In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
# 引入 LangChain 的输出解析器
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

# 1. 强制覆盖并加载环境变量
load_dotenv(override=True)

# 2. 定义大模型 (Language Model)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0) # 温度设为0，让它像个严谨的机器

# 3. 极度强大的 Output Parser：定义我们想要的结构
# 假设你在读 PDF 论文，你想提取它的核心算法和结论
class PaperSummary(BaseModel):
    algorithm_name: str = Field(description="论文中使用的主要算法名称")
    key_finding: str = Field(description="论文的最核心结论，限制在20个字以内")
    accuracy: float = Field(description="算法的准确率百分比，如 95.5")

parser = JsonOutputParser(pydantic_object=PaperSummary)

# 4. Model I/O 中的 Input：构建带格式化指令的 Prompt
# {format_instructions} 会自动把上面定义的格式要求塞进提示词里
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个极其严谨的科研论文数据提取专家。\n\n{format_instructions}"),
    ("user", "请提取以下论文摘要的信息：\n{paper_text}")
])

# 5. 见证 Chains 的威力：将 Prompt, 模型, 输出解析器串联
# 这里的 "|" 就是链式调用的灵魂
chain = prompt | llm | parser

# 6. 执行链！(未来这部分文本是由 Retrieval 从 PDF 里提取出来的)
dummy_pdf_text = """
在这项研究中，我们提出了一种名为深度神经网络强化检索(DNRR)的新架构。
该架构主要用于解决自然语言处理中的长文本依赖问题。
经过在多个公开数据集上的广泛测试，我们的模型最终达到了92.3%的准确率，
这证明了在缺乏高质量标注数据的情况下，DNRR依然能够保持鲁棒性。
"""

print("正在链式处理中...\n")
result = chain.invoke({
    "paper_text": dummy_pdf_text,
    "format_instructions": parser.get_format_instructions()
})

# 7. 打印最终结果（你会发现它是一个完美的 Python 字典，可以直接被代码调用！）
print(type(result))
print(result)

正在链式处理中...

<class 'dict'>
{'algorithm_name': '深度神经网络强化检索(DNRR)', 'key_finding': 'DNRR在缺乏标注数据时仍鲁棒', 'accuracy': 92.3}
